In [1]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv

# Carregar variáveis do arquivo .env
load_dotenv()

# Obter a API Key
api_key = os.getenv('TMDB_API_KEY')

# Verificar se a API Key foi carregada
if api_key:
    print("API Key carregada com sucesso!")
    # print(f"API Key: {api_key}") # Descomente para ver a chave, mas não deixe descomentado em código versionado
else:
    print("Erro: API Key não encontrada. Verifique seu arquivo .env")

# URL base da API v3 do TMDB
TMDB_API_BASE_URL = "https://api.themoviedb.org/3"

API Key carregada com sucesso!


In [2]:
# Endpoint de configuração
config_endpoint = f"{TMDB_API_BASE_URL}/configuration"

params = {
    'api_key': api_key
}

try:
    response = requests.get(config_endpoint, params=params)
    response.raise_for_status()  # Lança uma exceção para códigos de erro HTTP (4XX ou 5XX)

    config_data = response.json()
    print("Conexão bem-sucedida ao endpoint /configuration!")
    # print(json.dumps(config_data, indent=4)) # Imprime a configuração completa formatada

    # Vamos guardar a URL base para imagens, pode ser útil depois
    if 'images' in config_data and 'secure_base_url' in config_data['images']:
        secure_base_image_url = config_data['images']['secure_base_url']
        print(f"URL base segura para imagens: {secure_base_image_url}")
    else:
        print("Não foi possível obter a URL base para imagens da configuração.")

except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
    if response:
        print(f"Status Code: {response.status_code}")
        print(f"Response Text: {response.text}")

Conexão bem-sucedida ao endpoint /configuration!
URL base segura para imagens: https://image.tmdb.org/t/p/


In [4]:
genres_endpoint = f"{TMDB_API_BASE_URL}/genre/tv/list"

params_genres = {
    'api_key': api_key,
    'language': 'ko-KR' # Tenta obter em Coreano primeiro
}

drama_genre_id = None

try:
    response_genres = requests.get(genres_endpoint, params=params_genres)
    response_genres.raise_for_status()
    genres_data = response_genres.json()

    # print(json.dumps(genres_data, indent=4, ensure_ascii=False)) # Ver todos os gêneros em coreano

    # Procurar por "드라마" (Drama em coreano)
    for genre in genres_data.get('genres', []):
        if genre['name'] == '드라마': # "Drama" em coreano
            drama_genre_id = genre['id']
            print(f"ID do gênero '드라마' (Coreano): {drama_genre_id}")
            break
    
    if not drama_genre_id:
        print("Não encontrou '드라마'. Tentando em inglês 'Drama'...")
        params_genres['language'] = 'pt-BR' # Fallback para inglês
        response_genres_en = requests.get(genres_endpoint, params=params_genres)
        response_genres_en.raise_for_status()
        genres_data_en = response_genres_en.json()
        # print(json.dumps(genres_data_en, indent=4)) # Ver todos os gêneros em inglês

        for genre in genres_data_en.get('genres', []):
            if genre['name'] == 'Drama':
                drama_genre_id = genre['id']
                print(f"ID do gênero 'Drama' (Inglês): {drama_genre_id}")
                break
    
    if not drama_genre_id:
        print("Não foi possível encontrar o ID do gênero Drama.")

except requests.exceptions.RequestException as e:
    print(f"Erro na requisição de gêneros: {e}")
    if response_genres:
         print(f"Status Code: {response_genres.status_code}")
         print(f"Response Text: {response_genres.text}")

ID do gênero '드라마' (Coreano): 18


In [7]:
discover_endpoint = f"{TMDB_API_BASE_URL}/discover/tv"

# Parâmetros para a busca
# "Últimos 5 anos" = 2020-01-01 a 2024-12-31
first_air_date_gte = "2020-01-01"
first_air_date_lte = "2024-12-31"

params_discover = {
    'api_key': api_key,
    'with_original_language': 'ko',       # Coreano
    'with_genres': drama_genre_id,        # ID 18 para Drama
    'sort_by': 'popularity.desc',         # Mais populares primeiro
    'air_date.gte': first_air_date_gte,   # Data de primeira exibição A PARTIR DE
    'air_date.lte': first_air_date_lte,   # Data de primeira exibição ATÉ
    'language': 'pt-BR',                  # Tentar obter infos em Português do Brasil
    'include_adult': 'false',             # Não incluir conteúdo adulto
    'page': 1                             # Começar com a primeira página
}

print(f"Buscando Kdramas com os seguintes parâmetros:\n{json.dumps(params_discover, indent=2, ensure_ascii=False)}\n")

discovered_kdramas_page1 = []

try:
    response_discover = requests.get(discover_endpoint, params=params_discover)
    response_discover.raise_for_status()
    discover_data = response_discover.json()

    total_results = discover_data.get('total_results')
    total_pages = discover_data.get('total_pages')
    current_page = discover_data.get('page')
    
    print(f"Busca bem-sucedida!")
    print(f"Página atual: {current_page}")
    print(f"Total de resultados encontrados: {total_results}")
    print(f"Total de páginas: {total_pages}\n")

    results_on_page = discover_data.get('results', [])
    print(f"Encontrados {len(results_on_page)} resultados nesta página.\n")

    for kdrama in results_on_page[:5]: # Mostrar os primeiros 5 resultados da página
        print(f"  ID: {kdrama.get('id')}")
        print(f"  Título Original: {kdrama.get('original_name')}")
        print(f"  Título (PT-BR): {kdrama.get('name')}")
        print(f"  Data de Lançamento: {kdrama.get('first_air_date')}")
        print(f"  Nota Média: {kdrama.get('vote_average')}")
        print(f"  Popularidade: {kdrama.get('popularity')}")
        # print(f"  Sinopse (PT-BR): {kdrama.get('overview')}") # Pode ser longo
        print("-" * 20)
    
    # Guardar os resultados desta página
    discovered_kdramas_page1 = results_on_page

except requests.exceptions.RequestException as e:
    print(f"Erro na requisição de descoberta: {e}")
    if response_discover:
        print(f"Status Code: {response_discover.status_code}")
        print(f"Response Text: {response_discover.text}")

Buscando Kdramas com os seguintes parâmetros:
{
  "api_key": "733e5f49fab380dd108e4865e7b3f5c4",
  "with_original_language": "ko",
  "with_genres": 18,
  "sort_by": "popularity.desc",
  "air_date.gte": "2020-01-01",
  "air_date.lte": "2024-12-31",
  "language": "pt-BR",
  "include_adult": "false",
  "page": 1
}

Busca bem-sucedida!
Página atual: 1
Total de resultados encontrados: 803
Total de páginas: 41

Encontrados 20 resultados nesta página.

  ID: 93405
  Título Original: 오징어 게임
  Título (PT-BR): Round 6
  Data de Lançamento: 2021-09-17
  Nota Média: 7.862
  Popularidade: 65.4147
--------------------
  ID: 200709
  Título Original: 약한영웅
  Título (PT-BR): Classe dos Heróis Fracos
  Data de Lançamento: 2022-11-18
  Nota Média: 8.504
  Popularidade: 57.9219
--------------------
  ID: 112888
  Título Original: 여신강림
  Título (PT-BR): Beleza Verdadeira
  Data de Lançamento: 2020-12-09
  Nota Média: 8.277
  Popularidade: 41.0943
--------------------
  ID: 99966
  Título Original: 지금 우리 학교

In [8]:
if discovered_kdramas_page1:
    df_kdramas_page1 = pd.DataFrame(discovered_kdramas_page1)
    
    # Selecionar e renomear colunas de interesse
    columns_of_interest = {
        'id': 'ID_TMDB',
        'original_name': 'Título Original',
        'name': 'Título (PT-BR)',
        'first_air_date': 'Data Lançamento',
        'vote_average': 'Nota Média',
        'vote_count': 'Qtd Votos',
        'popularity': 'Popularidade',
        'overview': 'Sinopse (PT-BR)',
        'genre_ids': 'IDs Gêneros' # Lista de IDs de todos os gêneros do Kdrama
    }
    
    # Filtrar para manter apenas colunas existentes no DataFrame e que são de interesse
    existing_columns_of_interest = {k: v for k, v in columns_of_interest.items() if k in df_kdramas_page1.columns}
    df_kdramas_page1_display = df_kdramas_page1[list(existing_columns_of_interest.keys())].copy()
    df_kdramas_page1_display.rename(columns=existing_columns_of_interest, inplace=True)
    
    print(f"\nPrimeiros {len(df_kdramas_page1_display)} Kdramas da página 1 em um DataFrame:")
    display(df_kdramas_page1_display.head())
else:
    print("Nenhum Kdrama encontrado na primeira página para exibir no DataFrame.")


Primeiros 20 Kdramas da página 1 em um DataFrame:


,ID_TMDB,Título Original,Título (PT-BR),Data Lançamento,Nota Média,Qtd Votos,Popularidade,Sinopse (PT-BR),IDs Gêneros
0,93405,오징어 게임,Round 6,2021-09-17,7.862,15813,65.4147,Centenas de jogadores falidos aceitam um estra...,"[10759, 9648, 18]"
1,200709,약한영웅,Classe dos Heróis Fracos,2022-11-18,8.504,226,57.9219,"Com a ajuda de amigos inesperados, um aluno ta...","[10759, 18]"
2,112888,여신강림,Beleza Verdadeira,2020-12-09,8.277,2366,41.0943,"Im Ju-gyeong (Moon Ga-young), que alcançou a f...","[35, 18]"
3,99966,지금 우리 학교는,All of Us Are Dead,2022-01-28,8.288,4053,30.7107,Uma epidemia mortal surge em uma escola. Encur...,"[10759, 18, 10765]"
4,113268,경이로운 소문,Caçadores de Demônios,2020-11-28,8.323,365,28.9447,"De dia, trabalham num restaurante. À noite, sã...","[18, 9648, 10759]"


In [13]:
import time

# Parâmetros da Célula 4 ainda são válidos (discover_endpoint, api_key, etc.)
# Vamos reutilizar params_discover e o drama_genre_id

all_discovered_kdramas = []
MAX_PAGES_TO_FETCH = 5 # Defina um limite para não buscar tudo durante o teste inicial.
                       # Mude para 'total_pages' quando quiser buscar tudo.

print(f"Iniciando a coleta de Kdramas de múltiplas páginas...")

# Precisamos obter 'total_pages' da primeira chamada novamente, ou usar o valor já obtido
# Para segurança, faremos a primeira chamada aqui para garantir que temos o total_pages atualizado
# e não depender de uma variável de célula anterior que pode não ter sido executada.

params_discover['page'] = 1 # Garantir que começamos na página 1
try:
    response_initial_discover = requests.get(discover_endpoint, params=params_discover)
    response_initial_discover.raise_for_status()
    initial_discover_data = response_initial_discover.json()
    
    actual_total_pages = initial_discover_data.get('total_pages', 1)
    total_results_api = initial_discover_data.get('total_results', 0)
    
    print(f"API reporta um total de {total_results_api} resultados em {actual_total_pages} páginas.")

    # Decidir quantas páginas buscar: o mínimo entre MAX_PAGES_TO_FETCH e actual_total_pages
    # Cuidado: a API do TMDB limita a paginação a 500 páginas para /discover
    # Veja: https://developer.themoviedb.org/docs/errors -> "page_limit_exceeded"
    pages_to_fetch = min(MAX_PAGES_TO_FETCH, actual_total_pages, 500) 
    print(f"Vamos buscar até {pages_to_fetch} páginas (limite de 500 da API para /discover).")

    # Adicionar os resultados da primeira página já buscada
    results_page_1 = initial_discover_data.get('results', [])
    all_discovered_kdramas.extend(results_page_1)
    print(f"Página 1: {len(results_page_1)} resultados coletados.")

    # Loop para as páginas restantes
    for page_num in range(2, pages_to_fetch + 1):
        params_discover['page'] = page_num
        
        print(f"Buscando página {page_num}...")
        response_page = requests.get(discover_endpoint, params=params_discover)
        response_page.raise_for_status()
        page_data = response_page.json()
        
        results_on_page = page_data.get('results', [])
        all_discovered_kdramas.extend(results_on_page)
        print(f"Página {page_num}: {len(results_on_page)} resultados coletados. Total até agora: {len(all_discovered_kdramas)}")
        
        # IMPORTANTE: Respeitar o Rate Limit da API
        time.sleep(0.3) # Pausa de 300ms entre as requisições (ajuste conforme necessário)

    print(f"\nColeta finalizada. Total de {len(all_discovered_kdramas)} Kdramas coletados de {pages_to_fetch} páginas.")

except requests.exceptions.RequestException as e:
    print(f"Erro durante a coleta de múltiplas páginas: {e}")
    if 'response_initial_discover' in locals() and response_initial_discover:
        print(f"Status Code (initial): {response_initial_discover.status_code}")
        print(f"Response Text (initial): {response_initial_discover.text}")
    elif 'response_page' in locals() and response_page:
        print(f"Status Code (page): {response_page.status_code}")
        print(f"Response Text (page): {response_page.text}")

Iniciando a coleta de Kdramas de múltiplas páginas...
API reporta um total de 803 resultados em 41 páginas.
Vamos buscar até 5 páginas (limite de 500 da API para /discover).
Página 1: 20 resultados coletados.
Buscando página 2...
Página 2: 20 resultados coletados. Total até agora: 40
Buscando página 3...
Página 3: 20 resultados coletados. Total até agora: 60
Buscando página 4...
Página 4: 20 resultados coletados. Total até agora: 80
Buscando página 5...
Página 5: 20 resultados coletados. Total até agora: 100

Coleta finalizada. Total de 100 Kdramas coletados de 5 páginas.


In [11]:
if all_discovered_kdramas:
    df_all_kdramas = pd.DataFrame(all_discovered_kdramas)
    
    # Reutilizar a lógica de seleção e renomeação de colunas da Célula 5
    # existing_columns_of_interest foi definido na Célula 5
    # Se você não executou a Célula 5 ou limpou as variáveis, defina columns_of_interest novamente:
    columns_of_interest = {
        'id': 'ID_TMDB',
        'original_name': 'Título Original',
        'name': 'Título (PT-BR)',
        'first_air_date': 'Data Lançamento',
        'vote_average': 'Nota Média',
        'vote_count': 'Qtd Votos',
        'popularity': 'Popularidade',
        'overview': 'Sinopse (PT-BR)',
        'genre_ids': 'IDs Gêneros'
    }
    current_columns_in_df = {k: v for k, v in columns_of_interest.items() if k in df_all_kdramas.columns}
    df_all_kdramas_display = df_all_kdramas[list(current_columns_in_df.keys())].copy()
    df_all_kdramas_display.rename(columns=current_columns_in_df, inplace=True)
    
    print(f"\nDataFrame com {len(df_all_kdramas_display)} Kdramas coletados de {pages_to_fetch} páginas:") # pages_to_fetch deve estar definida da célula anterior
    display(df_all_kdramas_display.head())
    display(df_all_kdramas_display.tail())
    print(f"Dimensões do DataFrame: {df_all_kdramas_display.shape}")
else:
    print("Nenhum Kdrama foi coletado de múltiplas páginas.")


DataFrame com 100 Kdramas coletados de 5 páginas:


,ID_TMDB,Título Original,Título (PT-BR),Data Lançamento,Nota Média,Qtd Votos,Popularidade,Sinopse (PT-BR),IDs Gêneros
0,93405,오징어 게임,Round 6,2021-09-17,7.862,15813,65.4147,Centenas de jogadores falidos aceitam um estra...,"[10759, 9648, 18]"
1,200709,약한영웅,Classe dos Heróis Fracos,2022-11-18,8.504,226,57.9219,"Com a ajuda de amigos inesperados, um aluno ta...","[10759, 18]"
2,112888,여신강림,Beleza Verdadeira,2020-12-09,8.277,2366,41.0943,"Im Ju-gyeong (Moon Ga-young), que alcançou a f...","[35, 18]"
3,99966,지금 우리 학교는,All of Us Are Dead,2022-01-28,8.288,4053,30.7107,Uma epidemia mortal surge em uma escola. Encur...,"[10759, 18, 10765]"
4,113268,경이로운 소문,Caçadores de Demônios,2020-11-28,8.323,365,28.9447,"De dia, trabalham num restaurante. À noite, sã...","[18, 9648, 10759]"


,ID_TMDB,Título Original,Título (PT-BR),Data Lançamento,Nota Média,Qtd Votos,Popularidade,Sinopse (PT-BR),IDs Gêneros
95,135340,해피니스,Felicidade,2021-11-05,8.066,506,7.7943,Um novo tipo de vírus mortal se espalhou por t...,"[18, 9648, 10765]"
96,202318,일타 스캔들,Intensivão de Amor,2023-01-14,8.200,100,7.7843,Uma mãe generosa enfrenta uma competição acirr...,"[18, 35]"
97,203164,힘쎈여자 강남순,Strong Girl Nam-soon,2023-10-07,7.713,122,7.7742,De volta à Coreia em busca da família biológic...,"[35, 18, 10765, 80]"
98,214422,진짜가 나타났다!,Chegou a Hora da Verdade!,2023-03-25,7.400,18,7.7190,"Prestes a ser mãe solo, uma mulher bem-sucedid...",[18]
99,216310,무인도의 디바,Diva à Deriva,2023-10-28,8.200,58,7.6547,Um drama que retrata o desafio de diva de Seo ...,"[35, 18]"


Dimensões do DataFrame: (100, 9)


In [14]:
# Vamos pegar o ID do primeiro Kdrama da nossa lista para testar
if not df_all_kdramas.empty:
    sample_kdrama_id = df_all_kdramas.iloc[0]['id'] # Pega o ID da coluna 'id' original
    sample_kdrama_title = df_all_kdramas_display.iloc[0]['Título Original']
    print(f"Buscando detalhes para o Kdrama ID: {sample_kdrama_id} ({sample_kdrama_title})\n")

    # 1. Detalhes da Série (/tv/{series_id})
    tv_details_endpoint = f"{TMDB_API_BASE_URL}/tv/{sample_kdrama_id}"
    params_details = {
        'api_key': api_key,
        'language': 'pt-BR',
        'append_to_response': 'keywords,watch/providers' # Opcional: anexa palavras-chave e onde assistir
    }

    print(f"--- Detalhes da Série (endpoint: {tv_details_endpoint}) ---")
    try:
        response_details = requests.get(tv_details_endpoint, params=params_details)
        response_details.raise_for_status()
        details_data = response_details.json()

        # print(json.dumps(details_data, indent=4, ensure_ascii=False)) # Imprime todos os detalhes

        print(f"  Título Original: {details_data.get('original_name')}")
        print(f"  Título (PT-BR): {details_data.get('name')}")
        print(f"  Nº de Temporadas: {details_data.get('number_of_seasons')}")
        print(f"  Nº de Episódios: {details_data.get('number_of_episodes')}")
        print(f"  Status: {details_data.get('status')}")
        print(f"  Tagline: {details_data.get('tagline')}")
        
        genres_list = [genre['name'] for genre in details_data.get('genres', [])]
        print(f"  Gêneros: {', '.join(genres_list)}")
        
        prod_companies_list = [comp['name'] for comp in details_data.get('production_companies', [])]
        print(f"  Produtoras: {', '.join(prod_companies_list)}")

        networks_list = [net['name'] for net in details_data.get('networks', [])]
        print(f"  Emissoras Originais: {', '.join(networks_list)}")

        # Informações de "Onde Assistir" (se disponíveis para pt-BR)
        watch_providers_br = details_data.get('watch/providers', {}).get('results', {}).get('BR', {})
        if watch_providers_br:
            print(f"  Onde Assistir (BR):")
            if 'flatrate' in watch_providers_br:
                flatrate_providers = [p['provider_name'] for p in watch_providers_br['flatrate']]
                print(f"    Streaming: {', '.join(flatrate_providers)}")
            if 'buy' in watch_providers_br:
                buy_providers = [p['provider_name'] for p in watch_providers_br['buy']]
                print(f"    Comprar: {', '.join(buy_providers)}")
            if 'rent' in watch_providers_br:
                rent_providers = [p['provider_name'] for p in watch_providers_br['rent']]
                print(f"    Alugar: {', '.join(rent_providers)}")
        else:
            print(f"  Onde Assistir (BR): Informação não disponível.")


    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar detalhes da série: {e}")
        if 'response_details' in locals() and response_details:
            print(f"Status Code: {response_details.status_code}")
            print(f"Response Text: {response_details.text}")

    print("\n")

    # 2. Créditos da Série (/tv/{series_id}/credits)
    tv_credits_endpoint = f"{TMDB_API_BASE_URL}/tv/{sample_kdrama_id}/credits"
    params_credits = {
        'api_key': api_key
        # 'language': 'pt-BR' # Opcional para nomes de personagens, se traduzidos
    }

    print(f"--- Créditos da Série (endpoint: {tv_credits_endpoint}) ---")
    try:
        response_credits = requests.get(tv_credits_endpoint, params=params_credits)
        response_credits.raise_for_status()
        credits_data = response_credits.json()

        # print(json.dumps(credits_data, indent=4, ensure_ascii=False)) # Imprime todos os créditos

        # Elenco (primeiros 5)
        cast = credits_data.get('cast', [])
        print("  Elenco (Top 5):")
        for actor in cast[:5]:
            print(f"    - {actor.get('name')} como {actor.get('character')}")

        # Equipe (Diretores e Roteiristas)
        crew = credits_data.get('crew', [])
        directors = [member['name'] for member in crew if member.get('job') == 'Director']
        writers = [member['name'] for member in crew if member.get('department') == 'Writing' and 'Writer' in member.get('job', '')] # Pode haver vários tipos de "Writer"
        
        print(f"  Diretores: {', '.join(list(set(directors)))}") # set() para evitar duplicatas se houver
        print(f"  Roteiristas: {', '.join(list(set(writers)))}")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar créditos da série: {e}")
        if 'response_credits' in locals() and response_credits:
            print(f"Status Code: {response_credits.status_code}")
            print(f"Response Text: {response_credits.text}")
else:
    print("DataFrame df_all_kdramas está vazio. Execute as células anteriores para populá-lo.")

Buscando detalhes para o Kdrama ID: 93405 (오징어 게임)

--- Detalhes da Série (endpoint: https://api.themoviedb.org/3/tv/93405) ---
  Título Original: 오징어 게임
  Título (PT-BR): Round 6
  Nº de Temporadas: 3
  Nº de Episódios: 22
  Status: Returning Series
  Tagline: 45,6 bilhões de won é brincadeira de criança.
  Gêneros: Action & Adventure, Mistério, Drama
  Produtoras: Siren Pictures, Firstman Studio
  Emissoras Originais: Netflix
  Onde Assistir (BR):
    Streaming: Netflix, Netflix Standard with Ads


--- Créditos da Série (endpoint: https://api.themoviedb.org/3/tv/93405/credits) ---
  Elenco (Top 5):
    - Lee Jung-jae como Seong Gi-hun / 'No. 456'
    - Wi Ha-jun como 
    - Lee Byung-hun como 
    - Yim Si-wan como 
  Diretores: 
  Roteiristas: 


In [1]:
# No seu ambiente Python local/WSL com pandas e pyarrow instalados
import pandas as pd
try:
    df_silver = pd.read_parquet('data/silver/kdramas_silver.parquet') # Ajuste o caminho se executar de fora da raiz do projeto
    print("--- Info Camada Silver ---")
    df_silver.info(verbose=True, show_counts=True)
    print("\n--- Amostra Camada Silver (head) ---")
    print(df_silver.head())
    # Verifique colunas específicas, por exemplo, de listas:
    # print("\n--- Exemplo coluna 'genres' ---")
    # print(df_silver['genres'].head())
except Exception as e:
    print(f"Erro ao ler o arquivo Parquet da Camada Silver: {e}")

Erro ao ler o arquivo Parquet da Camada Silver: [Errno 2] No such file or directory: 'data/silver/kdramas_silver.parquet'
